# sqlalchemy私房手册-core进阶

## engine和connections

### 基本用法

In [2]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///localhost/core.db')

engine和DBAPI的connect函数并不同义，connect仅代表一次连接。而engine应该在应用程序的模块级别创建一次是最有效的，而不是每个对象或者每个函数创建。
```python

from sqlalchemy import text

with engine.connect() as connection:
    result = connection.execute(text("select username from users"))
    for row in result:
        print("username:", row['username'])
```
`engine.connect()`创建的connection对象是DBAPI连接的一个代理对象。`execute`方法返回一个`CursorResult`对象，它引用了DBAPI的游标对象。这里注意两点：
1. DBAPI游标将被`CursorResult`关闭，当它的所有结果行(如果有的话)被耗尽。当`CursorResult`没有返回行，则在创建的时候就会释放游标资源。
2. 当`connection.close()`调用，引用的DBAPI连接被释放到连接池，从数据库的角度来看，如果连接池还有空间存储这个连接以供下次使用，那么连接池实际上不会关闭连接。当连接被返回到池中供重用时，池机制会对DBAPI连接发出`rollback()`调用，删除任何事务状态或锁，并且连接为下一次使用做好准备。

`rollback`调用不是所有操作都回退了吗？这里没太理解。

### 使用事务

不象orm的session自动启用事务，在core中，需要手动的启用事务，有两种启用方法：
- 方法一：
```python
with engine.connect() as connection:
    with connection.begin():
        r1 = connection.execute(table1.select())
        connection.execute(table1.insert(), {"col1": 7, "col2": "this is some data"})
```
- 方法二：
```python
with engine.begin() as connection:
    r1 = connection.execute(table1.select())
    connection.execute(table1.insert(), {"col1": 7, "col2": "this is some data"})
```

使用with上下文管理器，在抛出异常时会自动的回退，如果没有异常则会自动的提交。`connection.begin()`返回的实际上是一个`transaction`对象，具有`rollback`和`commit`方法。